In [43]:
import deeplake
import model
ds = deeplake.load("hub://activeloop/kth-actions")

-

Opening dataset in read-only mode as you don't have write permissions.


/

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/kth-actions



|

hub://activeloop/kth-actions loaded successfully.



In [51]:
 ! wget http://www.nada.kth.se/cvap/actions/walking.zip

--2024-05-09 18:14:29--  http://www.nada.kth.se/cvap/actions/walking.zip
Resolving www.nada.kth.se (www.nada.kth.se)... 130.237.28.41, 2001:6b0:1:11c2::82ed:1c29
Connecting to www.nada.kth.se (www.nada.kth.se)|130.237.28.41|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.nada.kth.se/cvap/actions/walking.zip [following]
--2024-05-09 18:14:30--  https://www.nada.kth.se/cvap/actions/walking.zip
Connecting to www.nada.kth.se (www.nada.kth.se)|130.237.28.41|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19248 (19K) [text/html]
Saving to: ‘walking.zip’

walking.zip         100%[===================>]  18.80K  --.-KB/s    in 0s      

2024-05-09 18:14:32 (134 MB/s) - ‘walking.zip’ saved [19248/19248]



In [49]:
dataloader = ds.pytorch(num_workers=0, batch_size=4, shuffle=False) #, transform = {'videos': trsfm, 'action': None})
# ds.visualize()

In [50]:
for (image, action) in list(enumerate(dataloader))[:10]:
    print(action)

RuntimeError: stack expects each tensor to be equal size, but got [555, 120, 160, 3] at entry 0 and [675, 120, 160, 3] at entry 1

In [31]:
from torchvision import datasets, transforms

trsfm = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((40,60))
])

In [37]:
from torch.utils.data import DataLoader, Dataset
import numpy as np

class ClassificationDataset(Dataset):
    def __init__(self, ds, transform = None):
        self.ds = ds
        self.transform = transform

    def __len__(self):
        return len(self.ds)
    
    def __getitem__(self, idx):
        image = self.ds.videos[idx].numpy()
        label = self.ds.action[idx].numpy(fetch_chunks = True).astype(np.int32)

        if self.transform is not None:
            image = self.transform(image)

        sample = {"images": image, "action": label}

        return sample

In [38]:
kth_dataset = ClassificationDataset(ds)

In [41]:
kth_dataset['action']

InvalidKeyTypeError: Item 'action' of type 'str' is not a valid key.

### LABELS

In [18]:
ds.action[100].numpy()[0]

1

### Getting a 100 frames from index 100 to 200

In [22]:
!pip3 install av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 3.4 MB/s eta 0:00:00m eta 0:00:010:01:01mm


In [24]:
video = ds.videos[1, 100:200].numpy()
video.shape

(100, 120, 160, 3)

In [26]:
data = ds.videos[1, 15:20].data()
print(data['frames'].shape, data['timestamps'])

(5, 120, 160, 3) [0.6  0.64 0.68 0.72 0.76]


In [27]:
video = ds.videos[1].numpy()
video.shape

(675, 120, 160, 3)